In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import brickschema
from brickschema.namespaces import BRICK, RDFS, OWL, TAG, RDF, SH
from rdflib import URIRef
import networkx

from utils import *
from rules.relationship_relaxation import *
from rules.type_relaxation import *
from relaxation_graphs.full_relaxation_graph import *

In [ ]:
def create_ordered_list(triples_copy):
    ordered_list = []
    for item in triples_copy:
        for item_i in item:
            item_new = item_i.split('#')[-1]
            ordered_list.append(item_new)
    ordered_list.sort()
    return ordered_list

def generates_answer(triples_copy, file_input, profile_input, building_graph):
    ### all this can probably be done in main script, so we dont load each time
    g = brickschema.Graph(load_brick=True) 
    g.load_file(file_input) # "../Data - Brick models/msb.ttl"
    g.expand(profile=profile_input) # "owlrl"
    ### 
    
    # Make function to turn triples in list format back to string query format? triples_copy
    res = g.query(q)
    if res == True: # Something like this?? or if the res contains a length/output
        generates_answer = 'True'
    else:
        generates_answer = 'False'
        
    return generates_answer
        
                
def relax_triples(G, triples, rules, rule_names, level_itr, node_idx, k_node, building_graph):
    node_list =[]
    
    for r_index in range(len(rules)):
        rule = rules[r_index]
    
        for i in range(len(triples)):
            triple = triples[i]
            relaxed_triples = rule(triple)

            if len(relaxed_triples) > 0:
                for relaxed_triple in relaxed_triples:
                    triples_copy = triples.copy()
                    triples_copy[i] = relaxed_triple
                    
                    #Checking to see if it exists within graph
                    ordered_list = create_ordered_list(triples_copy)
                    # Checking to see if triple gnerates answer
                    answer = generates_answer(triples_copy, file_input, profile_input, building_graph)
                    
                    if any(G.nodes.data()[node]['ordered_id'] == ordered_list for node in G) == False:
                        if answer == 'True':
                            G.add_node(node_idx, query=triples_copy, 
                                       ordered_id = ordered_list, generates_answer=answer)
                            G.add_edge(k_node, node_idx, rule=rule_names[r_index], 
                                       level=level_itr, triple=i)
                            node_idx+=1
                        else: # answer == 'False':
                            G.add_node(node_idx, query=triples_copy, 
                                       ordered_id = ordered_list, generates_answer=answer)
                            G.add_edge(k_node, node_idx, rule=rule_names[r_index], 
                                       level=level_itr, triple=i)
                            # add that node with no generated answer to list of new nodes to be relaxed in next itr
                            node_list.append(node_idx)
                    else:
                        #print('node deleted')
                        pass
                        
    return G, node_idx, node_list

def relaxtion_graph(query, limit_level, building_graph):
    rules = [ApplyRule_LowerClass, ApplyRule_SiblingClass, ApplyRule_UpperClass, apply_rule_variable_relationship,
             apply_rule_transitive_relationship]
    rule_names = ['lower', 'sibling', 'upper', 'relationship', 'transitive_relationship']
    
    if limit_level == True:
        limit_num = int(input('Enter desired level of relaxation:- '))
    else:
        limit_num = 10000000
                        
    triples = extract_triples(query)

    G = networkx.Graph()
    node_idx = 1
    ordered_list = create_ordered_list(triples)
    # Orginal query will always have a false generates answer, thus the need to relax
    G.add_node(node_idx, query=triples, ordered_id = ordered_list, generates_answer='False')
    node_idx+=1
    G_old = G.copy()
        
    # Create First Layer, 
    #start_node = node_idx
    level_itr = 1
    G_new, node_idx, node_list = relax_triples(G, triples, rules, rule_names, level_itr, node_idx, k_node=1, , building_graph)
    print('Level '+ str(level_itr) + ' complete')
    #end_node = node_idx
    G_new = G.copy()
    # List of all node indexes with False 'generates_answer' attribute

    # While layer contains False 'generates_answer' node(s), continue
    while len(node_list) > 0:
        level_itr += 1
        for k_node in node_list:
            query = G.nodes.data()[k_node]['query']
            triples = query.copy()
            G_new, node_idx, node_list = relax_triples(G, triples, rules, rule_names, level_itr, node_idx, k_node)
        print('Level '+ str(level_itr) + 'complete')
        if level_itr > limit_num:
            break

    return G_new

In [ ]:

def main(query, limit_level=False, export=False, building_graph):
    G_relaxed = relaxtion_graph(query, limit_level, building_graph)
    print('Relaxed query graph generated')
    if export == True:
        # saving graph created above in gexf format, can be visualized using Gephi
        nx.write_gexf(G_relaxed, "full_relaxed_query.gexf")
        print('GEXF file generated')
        
    return G_relaxed


In [4]:
q = """SELECT ?vav WHERE  {
    ?vav a brick:VAV
}"""

In [6]:
g = brickschema.Graph(load_brick=True)
g.load_file("Data - Brick models/artx.ttl")
g.expand(profile="owlrl")

run_brick_query(building_model=g, query=q)

[rdflib.term.URIRef('http://buildsys.org/ontologies/ARTX#VAVRM120'),
 rdflib.term.URIRef('http://buildsys.org/ontologies/ARTX#VAVRM100'),
 rdflib.term.URIRef('http://buildsys.org/ontologies/ARTX#VAVRM112'),
 rdflib.term.URIRef('http://buildsys.org/ontologies/ARTX#VAVRM107B'),
 rdflib.term.URIRef('http://buildsys.org/ontologies/ARTX#VAVRM110'),
 rdflib.term.URIRef('http://buildsys.org/ontologies/ARTX#VAVRM103'),
 rdflib.term.URIRef('http://buildsys.org/ontologies/ARTX#VAVRM107A'),
 rdflib.term.URIRef('http://buildsys.org/ontologies/ARTX#VAVRM115')]

In [6]:
op = []
for row in res:
    print(row)
    op.append(row[0])
    
print(op)

(rdflib.term.URIRef('http://buildsys.org/ontologies/MSB#MSB.ZONE.AHU05.RM060.Zone_Supply_Air_Flow'), rdflib.term.URIRef('http://buildsys.org/ontologies/MSB#VAVRM060'))
[rdflib.term.URIRef('http://buildsys.org/ontologies/MSB#MSB.ZONE.AHU05.RM060.Zone_Supply_Air_Flow')]
